# Tutorial on Speech

In this tutorial we will investigate the representation of speech using audio signals and spectrograms, before making use of trained speech to text models. Finally we will make use of text to speech models to generate new audio.

**Optional for Colab users**

Before starting, we can set up the connection with the Google Dive storage, to keep there our documents.
Just execute the following passages:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Make sure that the variable path contains the correct sequence of folders separate by a `'/'` to get to your lecture files

In [ ]:

import os

path = 'Colab Notebooks/NLP/Practical_11__Speech'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

## 1.0 Representing Audio Signals
In this section we will investigate audio signals and their various representations. (This part of the tutorial is based on code written by Dr Vincenzo Scotti.)

### 1.1 Loading and Visualising a Signal

We'll start by loading the audio signal in the WAV file that came with the tutorial. Upload the file to Google colab and then load it as follows:

In [ ]:
import librosa

file_path = 'audio.wav'
audio, sr = librosa.load(file_path, sr=16000)
audio

We see that the audio signal is just a big array of floats, repersenting a time series of pressure values. The librosa library that we used to load the signal has resampled the signal at a sampling rate we specified with the parameter 'sr'.

We can visualise the time series as a graph:

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 12}) ## make the font of the axes ticks/labels a bit bigger

fig = plt.figure(figsize=(16,4))
plt.plot(audio, linewidth=0.4)
plt.ylabel('Amplitude')
plt.xlabel('Samples')
plt.show()

We see a typical time series for speech.
The x-axis is the sample and the y-axis is the amplitude of the signal.

We can change the x-axis to be time once we know the duration of the time series, which we can get as follows:

In [ ]:
duration = librosa.get_duration(y=audio, sr=sr)
duration

So the audio clip runs for just over 4 seconds.

We can now regenerate the graph with the x-axis being time in seconds:

In [ ]:
import numpy as np
num_samples = audio.shape[0]
time = np.linspace(0, duration, num_samples)

fig = plt.figure(figsize=(16,4))
plt.plot(time, audio, linewidth=0.4)
plt.ylabel('Amplitude')
plt.xlabel('Time [s]')
plt.show()

We can listen to the audio signal in colab as follows:

In [ ]:
from IPython.display import Audio
Audio(audio, rate=sr)

### 1.2 Representing the signal in the Frequency domain

When representing audio signals we often convert them to the frequency domain using the Fast Fourier Transform algorithm. We'll do that now to have a look at the signal in terms of its frequencies. (Note we'll take a small slice of the signal to do this.)

In [ ]:
import math

w = int(math.ceil(0.025 * sr)) # use a window of size 25ms
d = int(math.ceil(0.01 * sr)) # and a delay between consecutive windows of 10ms
t = int(math.ceil(1.2 * sr))

n_fft = 512 # use a Fast Fourier transform of size 512 samples
spec = librosa.stft(y=audio, n_fft=n_fft, win_length=w, hop_length=d,)
freq = np.linspace(0,sr/2,spec.shape[0])
s = int(math.ceil(t/d))

fig = plt.figure(figsize=(8,4))
plt.plot(freq, np.abs(spec)[:, s])
plt.ylabel('Magnitude')
plt.xlabel('Frequency [Hz]')
plt.xlim([freq[0], freq[-1]])
plt.ylim([-1, 15])
plt.grid()
plt.tight_layout()
plt.show()

Typically we represent the importance of each frequency by the number of decibells, which is related to the logarithm of the amplitute of each frequency, (more precisily it is 10 times the logarithm of the squared amplitude of the signal, which is the engergy of the signal).

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.plot(freq, librosa.power_to_db(np.abs(spec) ** 2, ref=np.max)[:, s])
plt.ylabel('Power [dB]')
plt.xlabel('Frequency [Hz]')
plt.xlim([freq[0], freq[-1]])
plt.ylim([-80, 0])
plt.grid()
plt.tight_layout()
plt.show()

Note that the higher frequency components of the signal are then highlighted by the decibel conversion.

Speech consists of many different sounds one after another. So the typical representation of a speech signal is to divide the original signal up into short overlapping segments and then calculate the frequency distribution for each. Each segment is usually smoothed with a (Hann) windowing function. We can visualise this process as follows. First let's define the window size, delay and start time:

In [ ]:
import math

w = int(math.ceil(0.025 * sr)) # use a window of size 25ms
d = int(math.ceil(0.01 * sr)) # and a delay between consecutive windows of 10ms
t = int(math.ceil(1.2 * sr)) # start at 1.2 seconds into the signal

Now let's plot a small segment of the audio signal with the windowing applied:

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(16, 8), sharex=True, sharey=True)
axes[0].plot(time[t:t+(2*w)], audio[t:t+(2*w)])
axes[0].set_ylabel('Amplitude')
axes[0].set_xlim([time[t], time[t+(2*w)]])
axes[0].grid(True)

for i in range(3):
    axes[i + 1].plot(time[t+(i*d):t+(i*d)+w], audio[t+(i*d):t+(i*d)+w], c='tab:blue', label='Speech segment')
    axes[i + 1].plot(time[t+(i*d):t+(i*d)+w], audio[t+(i*d):t+(i*d)+w] * librosa.filters.get_window('hann', w), c='tab:red', label='Hann windowed')
    axes[i + 1].set_ylabel('Amplitude')
    axes[i + 1].grid(True)
    axes[i + 1].legend(loc='best')

axes[3].set_xlabel('Time [s]')
plt.show()

Having seen the visualisation, and understanding the process by which a Spectrogram is created, we can now directly call a method that will create the Spectrogram.

In [ ]:
n_fft = 512 # use a Fast Fourier transform of size 512 samples
spec = librosa.stft(y=audio, n_fft=n_fft, win_length=w, hop_length=d,)
spec.shape

So now that we have converted a one-dimensional time series (audio signal) into a 2 dimenionsional matrix (spectrogram).
We can visualise the spectrogram as follows:

In [ ]:
freq = np.linspace(0, sr / 2, spec.shape[0])
spec_time = np.linspace(0, duration, spec.shape[1])

fig = plt.figure(figsize=(10,4))
plt.pcolormesh(spec_time, freq, librosa.power_to_db(np.abs(spec) ** 2, ref=np.max))
plt.colorbar(label='Power [dB]')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.tight_layout()
plt.show()

### 1.3 The Mel spectrogram

The spectrogram shown above used a linear y-axis for the frequency dimension. We humans are sensitive to relative changes in pitch (which correspond to multiplicative changes in frequency) rather than absolute (additive) changes in the pitch.

So in Mel (short for melody) spectrogram, we transform the y-axis onto a logarithmic scale, using a mapping to compute mel frequencies as follows:

In [ ]:
n_mel = 80
lin_freq = librosa.mel_frequencies(n_mels=n_mel, fmax=sr / 2)

fig = plt.figure(figsize=(16,4))
plt.plot(lin_freq, np.arange(n_mel))
plt.ylabel('Mel frequency')
plt.xlabel('Linear frequency [Hz]')
plt.xlim([0, lin_freq[-1]])
plt.ylim([0, 80])
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, win_length=w, hop_length=d, n_mels=n_mel)
mel_spec.shape

In this way, when we plot the spectrogram the high frequencies are compressed and the lower frequencies expanded along the y-axis, such that the each step along the axis corresponds to the same multiplicative increase in the frequency:

In [ ]:
mel_comp = np.arange(mel_spec.shape[0]) + 1
mel_spec_time = np.linspace(0, duration, mel_spec.shape[1])

fig = plt.figure(figsize=(10,4))
plt.pcolormesh(mel_spec_time, mel_comp, librosa.power_to_db(mel_spec, ref=np.max))
plt.colorbar(label='Power [dB]')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.yticks(ticks=mel_comp[9::10], labels=[f'{freq:.0f}' for freq in lin_freq[9::10]])
plt.tight_layout()
plt.show()

### 1.4 Record voice

Let's now record our own voice and create a Mel spectrogram from it.

To record audio we'll make use of code to from here: https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb

In [ ]:
!pip install ffmpeg-python

In [ ]:
"""
Here are the references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
num_samples = audio.shape[0]
time = np.linspace(0, duration, num_samples)

fig = plt.figure(figsize=(16,4))
plt.plot(time, audio, linewidth=0.4)
plt.ylabel('Amplitude')
plt.xlabel('Time [s]')
plt.show()

We can save this audio recording for later use ...

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

Copy code from above to generate the Mel Spectrogram for your own voice.

## 2.0 Speech Recognition

Now that we know how to represent audio signals in a way that highlights the different components of the speech, we can try to make use of techniques for recognising speech sounds in the signal.

We will make use of a state-of-the-art transformer-based speech recognition system from OpanAI called Whisper: https://github.com/openai/whisper



Let's first install the Python library:

In [ ]:
!pip install -U openai-whisper

Whisper models come pretrained with various model sizes. We will download one of the smaller models, called "base", which is only 139MB. Feel free to try a larger model!

In [ ]:
import whisper

model = whisper.load_model("base")

Now that we have the model, we can use it for transcribing audio to text. Let's start with our original WAV file.

In [ ]:
result = model.transcribe("audio.wav")
print(result["text"])

If you see a warning (about FP16 not being supported) just ignore it. It's because you are running Whisper on the CPU rather than a GPU, but there's no need for a GPU for the moment.

How did it go? Did it transcribe the text correctly?
(If not, why not, and what could be done about it?)

### 2.1 Detecting the spoken language

The model we downloaded for Whisper can transcribe text not only in English but in MANY languages.

As part of the functionality Whisper can detect the spoken language, so let's try it.

We will now follow the pipeline for processing the audio that was hidden from us when we directly called the "transcribe()" method above. The first step is to detect the speech in the audio signal and trim or pad the signal to a certain length (30 seconds):

In [ ]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("audio.wav")
audio = whisper.pad_or_trim(audio)
audio

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(audio, linewidth=0.4)
plt.ylabel('Amplitude')
plt.xlabel('Samples')
plt.show()

The next step is to compute the Mel Spectrogram:

In [ ]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
mel

Let's visualise the spectrogram:

In [ ]:
fig = plt.figure(figsize=(10,4))
plt.pcolormesh(mel.cpu().numpy())
plt.colorbar(label='Power [dB]')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [10ms]')
plt.show()

Note that the signal has been padded out to 30 seconds.

Now let's use Whisper to detect the language based on the audio clip:

In [ ]:
_, probs = model.detect_language(mel)
lang = max(probs, key=probs.get)
print(f"Detected language: {lang}, confidence: {probs[lang]}")

So the model is pretty certain that the language is English!

Although it had quite a few languages to choose from. Have a look:

In [ ]:
probs

Try loading the second audio sample from the tutorial "italiano.mp3" and see if the model can correctly determine the language.

In [ ]:
audio2 = whisper.load_audio("italiano.mp3")

Audio(audio2, rate=sr)

What happened?? Try changing the sample rate (sr) to 16000. -- Whisper has resampled the audio at a new rate of 16kHz.

Any better?

In [ ]:
new_audio = Audio(audio2, rate=16000)
new_audio

Now copy the code from above to predict the language:

In [ ]:
# TODO

Did it work? How confident was the model that the language was Italian?

Now we can try to transcribe the Italian audio:

In [ ]:
audio2 = whisper.pad_or_trim(audio2)
mel = whisper.log_mel_spectrogram(audio2).to(model.device)

# decode the audio
options = whisper.DecodingOptions(fp16 = False)
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)


How good is Whisper at understanding Fantozzi?

### 2.2 Transcribing own audio

Try transcribing the audio that you produced before. Does it work?

In [ ]:
result = model.transcribe("recording.wav")
print(result["text"])

## 3.0 Text to Speech


We'll make use of the Tacotron2 system following this notebook: https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/nvidia_deeplearningexamples_tacotron2.ipynb

To run the following code you will need to use a GPU, so change the runtime above to include one if you haven't already.

In [ ]:
!pip install numpy scipy librosa unidecode inflect
#!apt-get update
#!apt-get install -y libsndfile1

Load the Tacotron model from Torchhub:

In [ ]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Tacotron is a "Spectrogram predictor". It has been trained to input a text (or phonetic sequence) and output a mel spectrogram. A Vocoder is then needed to convert the spectrogram into an audio signal. In our case we will make use of the Waveglow vocoder for this purpose: https://github.com/NVIDIA/waveglow

Load the waveglow model as follows:

In [ ]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

Now we have all we need. Let's write some text that we can use to generate the audio:

In [ ]:
text = "Hello crazy world, I missed you so much."
text = "Wow. This NLP class is just so amaaaaaaazing. By far the best class I've ever taken at the Politecnico!"

That text is read in by the model and converted to a numeric sequence:

In [ ]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])
sequences

### 3.1 Producing a Spectrogram

Now convert the sequence into a mel spectrogram with Tacotron2

In [ ]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)

We now have a spectrogram:

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(10,4))
plt.pcolormesh(mel[0].cpu().numpy())
plt.colorbar(label='Power [dB]')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [10ms]')
plt.show()

### 3.2 Producing Audio

Now use Waveglow to produce the audio signal

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050
audio_numpy

Have a look at the audio signal:

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(audio_numpy, linewidth=0.4)
plt.ylabel('Amplitude')
plt.xlabel('Samples')
plt.show()

Let's listen to it!

In [ ]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

Why not save the audio and see if the speech-to-text system can recognise it?

In [ ]:
from scipy.io.wavfile import write
write("audio_tts.wav", rate, audio_numpy)

Want to try to generate other types of voices? Check out this code from Vincenzo Scotti: https://github.com/vincenzo-scotti/tts_mellotron_api

The END. Hope you liked the Speech tutorial!